In [2]:
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from app.utils import normalize_persian
from app.config import *
import os

def build_index():
    # ایجاد دایرکتوری‌های مورد نیاز
    os.makedirs(os.path.dirname(INDEX_PATH), exist_ok=True)
    os.makedirs(os.path.dirname(EMBEDDING_PATH), exist_ok=True)

    try:
        xls = pd.read_excel(DATA_PATH, engine='openpyxl')
        questions = xls["Q"].apply(normalize_persian).tolist()
        print(f"تعداد سوالات بارگیری شده: {len(questions)}")
    except Exception as e:
        print(f"خطا در خواندن فایل داده: {str(e)}")
        return

    try:
        model = SentenceTransformer(EMBEDDING_MODEL)
        embeddings = model.encode(questions, convert_to_numpy=True)
        np.save(EMBEDDING_PATH, embeddings)
        print(f"تعداد embedding های ایجاد شده: {embeddings.shape[0]}")
    except Exception as e:
        print(f"خطا در ایجاد embedding: {str(e)}")
        return

    try:
        dim = embeddings.shape[1]
        index = faiss.IndexFlatIP(dim)
        index.add(embeddings)
        faiss.write_index(index, INDEX_PATH)
        print(f"ایندکس با موفقیت ساخته و در {INDEX_PATH} ذخیره شد")
    except Exception as e:
        print(f"خطا در ساخت ایندکس: {str(e)}")

if __name__ == "__main__":
    build_index()

تعداد سوالات بارگیری شده: 53


D:\PycharmProj\QnA-Farsi\venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


تعداد embedding های ایجاد شده: 53
ایندکس با موفقیت ساخته و در D:\PycharmProj\QnA-Farsi\index\faiss_index.bin ذخیره شد
